In [1]:
import requests as r
import datetime as dt
import pandas as pd

# KiteUp - Icelandic weather alert system for kitesurfers

In [10]:

'''Constants
Weather Station Id's'''
REY = 1 # Reykjavík : Grótta
EYR = 1395 # Eyrarbakki : Ölfurárós
GEL = 1480 # Geldingarnes : Geldingarnes
STR = 1473 # Straumsvík : Skógstjörn, Grótta


'''Configure your variables:
hour_range - Hour range, compared with <= and >=
wind_range - In m/s, <= and >=
min_rows - Minimum rows per day. Some forecasts only predict every 3 hours, Some every hour. compared with <= and >=
locations - Put in the stations you want to get updates about along with the directions you want to filter for
            Each weather station can have many kiteable locations you can check for, see STR(Straumsvík) station for example.
'''

# Time range
hour_range = ['09:00','20:00']
# Wind with m/s range that will be compared with <= and >=
# A 9 metre kite your wind range could be around [6,12] m/s
wind_range = [6,12]

# For rain filter, checks if it contains any of these in a string lowercase
do_not_want = ['rign','skúr'] # Leave empty if you want any kind of weather condition


check_days = 3 # How many days ahead to check, including today

include_directions = True # if to include the directions for the spots given below
min_rows = 2 # Setting min rows as 2 because some forecasts only predict for every 3 hours

# Spots: Directions
# Try to come up with the best locations and directions to look for
GROTTA = {'Grótta':['N','NNA','NV','NNV','VNV','V','VSV','SV']}
SKOGTJ = {'Skógstjörn':['S','SA','SSA','SV','SSV','VSV']}
GELNES = {'Geldingarnes':['A','ASA','SSA','SA','V','VNV']}
OLFUS = {'Ölfusárlón':['A','ASA','SSA','S','SA','SSV','VSV','VNV','V']}

# Weather stations can have many spots
locations = {
                REY:[GROTTA],
                EYR:[OLFUS],
                GEL:[GELNES],
                STR:[SKOGTJ, 
                    GROTTA]
            }



In [11]:
def query_weather_api():
    base_url = "https://apis.is"
    path = "/weather/forecasts/is/?stations="
    station_text_format = "{}," * len(locations)
    stations = station_text_format.format(*locations.keys())
    res = r.get(url=base_url + path + stations)
    return res
    '''
    Response:
    results - listi af dict:
        id - Stöðvanúmer
        name - Nafn á stöð
        atime - tími hvenær spá er gefin út á forminu %Y-%m-%d %H:%M:%S
        forecast - list af dict: 
            ftime - tími á forminu %Y-%m-%d %H:%M:%S
            F - vindhraði, 
            D - vindstefnu'''

In [12]:
def wind_filter(df, wind_range):
    return df[(df.F  >= wind_range[0]) & (df.F <= wind_range[1])]

def directions_filter(df, directions): # Checks the 'D' column (Direction)
    return df[df.D.isin(directions)]

def hour_range_filter(df, hour_range):
    return df.between_time(*hour_range)

def rain_filter(df, wcond): # Checks the 'W' column (Weather)
    if not wcond:
        return df
    for cw in wcond: # use ~ for to reverse the booleans because we do not want it to contain
        df = df[~df.W.str.lower().str.contains(cw)]
    return df

In [13]:

'''
day_check function:
Filters out if the forecast of the day has the right wind and kite–able directions
Parameters:
df – df containing weather for one day
location_id - weather station id '''
def day_check(df, location_id, link, utgafutimi):
    for spot in locations[location_id]:
        for name, directions in spot.items():
            df = wind_filter(df, wind_range)
            df = rain_filter(df, do_not_want)
            
            if not df.empty:
                if include_directions:
                    df = directions_filter(df, directions)
                if df.shape[0] >= min_rows:
                    print(spot)
                    print("Spá gefin út: " + utgafutimi)
                    print(link)
                    print(df.drop('ftime',axis=1).head())


In [14]:
def main():
    query_response = query_weather_api()
    vedur = query_response.json()['results']
    for i in range(len(vedur)):
        location_id = int(vedur[i]['id'])
        link = vedur[i]['link']
        
        utgafutimi = vedur[i]['atime']
        df = pd.DataFrame(vedur[i]['forecast'])
        df['ftime'] = df['ftime'].astype('datetime64[s]')
        df['F'] = df['F'].astype('float')
        
        # Set the index to a DateTimeIndex so we can filter by hour
        df.set_index(pd.DatetimeIndex(df['ftime']), inplace=True)
        df = hour_range_filter(df, hour_range)
        # Get the first date
        day = df['ftime'].iloc[0]

        for i in range(0, check_days):
            df_day = df[df.ftime.dt.day == day.day]
            # Call day_check with a dataframe for each day
            day_check(df_day, location_id, link, utgafutimi)
            # Iterate to the next days
            day += pd.DateOffset(days=1)


In [15]:
if __name__ == '__main__':
    main()

{'Grótta': ['N', 'NNA', 'NV', 'NNV', 'VNV', 'V', 'VSV', 'SV']}
Spá gefin út: 2018-09-27 12:00:00
http://www.vedur.is/vedur/spar/stadaspar/hofudborgarsvaedid/#group=100&station=1
                       D    F    N    R  T  TD           W
ftime                                                     
2018-09-27 16:00:00    V  6.0  100  0.0  7  -1      Skýjað
2018-09-27 17:00:00  VNV  6.0   10  0.0  7  -1  Léttskýjað
{'Grótta': ['N', 'NNA', 'NV', 'NNV', 'VNV', 'V', 'VSV', 'SV']}
Spá gefin út: 2018-09-27 12:00:00
http://www.vedur.is/vedur/spar/stadaspar/hofudborgarsvaedid/#group=100&station=1
                       D     F    N    R  T TD                      W
ftime                                                                
2018-09-28 12:00:00   SV  11.0  100  0.6  9  7  Lítils háttar rigning
2018-09-28 13:00:00   SV  10.0  100  0.2  9  7  Lítils háttar rigning
2018-09-28 14:00:00   SV  10.0  100  0.4  9  7                Rigning
2018-09-28 15:00:00   SV  11.0  100  0.7  8  5            